# Enunciado


3. Usando a experiência obtida na resolução dos problemas 1 e 2, e usando, ao invés  do grupo abeliano multiplicativo $\,\mathbb{F}_p^\ast\,$,  o  grupo abeliano aditivo que usou na pergunta 2,   
    1. Construa ambas as versões  IND-CPA segura e IND-CCA segura do esquema de cifra ElGamal em curvas elípticas.
    2. Construa uma implementação em curvas elípticas de um protocolo autenticado de “Oblivious Transfer” $\,\kappa$-out-of-$n\,$.

-----------

## Exercício 3.a

### **Versão IND-CPA**

#### Geração de chaves

**Chave Privada (sk)**: Um número inteiro aleatório no intervalo $[1, L-1]$, onde $L$ é a ordem do subgrupo gerado pelo ponto base $P$. \
**Chave Pública (pk)**: Um ponto na curva elíptica, calculado como $p_k = s_k * P$

#### Cifra

- Escolher um número inteiro aleatório $\kappa$ no intervalo $[1, L-1]$.
- Calcular o ponto $C_1 = \kappa * P$.
- Converter a mensagem $m$ num ponto na curva elíptica (como no método de Koblitz).
- Calcular o ponto $C_2 = m + \kappa * p_k$.
- O texto cifrado é o par $(C_1, C_2)$

#### Decifra

- Usar a chave privada $s_k$ para calcular $m = C_2 - s_k * C_1$.
- Converter o ponto $m$ de volta para a mensagem original

#### Diferenças entre usa o grupo multiplicativo $F_p^*$ e usar um grupo abeliano

No Exercício 3.A, a cifra e decifra não são iguais ao Exercício 1.A, porque as operações matemáticas mudam:

- No grupo multiplicativo $F_p^∗$, usamos exponenciação e multiplicação modular.
- Nas curvas elípticas, usamos adição de pontos.

Portanto, a cifra e decifra no Exercício 3.A devem ser adaptadas para as operações no grupo aditivo da curva elíptica. Comparando-as:

| Operação                     | Grupo Multiplicativo $ \mathbb{F}_p^* $ | Curva Elíptica $ E $ |
|------------------------------|-------------------------------------------|-------------------------|
| **Geração de Chaves (pk)**         | $ g^s\mod p $                          | $ g \cdot s $         |
| **Cifra**                     | $ \gamma = g^\omega \mod p $            | $ \gamma = \omega \cdot g $ |
|                              | $ \kappa = (g^s)^\omega \mod p $        | $ \kappa = \omega \cdot S $  |
|                              | $ \delta = m \cdot \kappa \mod p $      | $ C = M + \kappa $           |
| **Decifra**                   | $ \kappa = \gamma^s \mod p $            | $ \kappa = s \cdot \gamma $  |
|                              | $ m = \delta \cdot \kappa^{-1} \mod p $ | $ M = C - \kappa $           |

fonte: https://crypto.stackexchange.com/questions/9987/elgamal-with-elliptic-curves

#### Conversão de pontos

Especialmente no contexto de curvas elípticas, a conversão de pontos é uma técnica utilizada para mapear mensagens (ou dados) em pontos específicos sobre a curva elíptica.
Para conseguir agregar o melhor dos dois mundos (curvas elípticas e ElGamal) precisamos de mapear mensagens em pontos e vice-versa.

Nesta secção apresentamos a metodologia de conversão que iremos utilizar:

Para mapear uma mensagem de $p−1−l$ bits num ponto $(x,y)$ sobre uma curva elíptica definida no corpo primo abeliano de $p$ bits fazemos passo a passo:

1. **Preparação da Mensagem:**

    - A mensagem $m$ tem $p−1−l$ bits.
    - Para ajustar o tamanho, a mensagem é concatenada com $l$ bits zero, resultando em $x=m∥0l$.

2. **Cálculo de $x'$:**

    - O valor $x'$ é calculado como $x' = x^3 + a⋅x +b \mod p$, onde $a$ e $b$ são parâmetros da curva elíptica, e $p$ é o primo que define o corpo finito.

3. **Verificação do Resíduo Quadrático**:
    
    - Se $x'$ for um resíduo quadrático módulo $p$, então existe um $y$ tal que $y^2≡x' \mod p$. Nesse caso, o ponto $(x,y)$ é um ponto válido na curva.
    - Se $x'$ não for um resíduo quadrático, os últimos $l$ bits de $x$ são incrementados em 1, e o processo é repetido.

4. **Limite de Tentativas:**

    - O processo é repetido até $2^l$ vezes. Se nenhum $x'$ válido for encontrado, a mensagem é considerada "não codificável".

5. **Decodificação:**

    - Para recuperar a mensagem original, basta ignorar a coordenada $y$ e remover os últimos $l$ bits da coordenada $x$.

fonte: https://crypto.stackexchange.com/questions/76340/how-to-create-an-ec-point-from-a-plaintext-message-for-encryption

#### **Implementação**

Usando a classe definida no exercício 2:

In [1]:
import hashlib
import random
from sage.all import *

class EcDSA_Ed25519:
    def __init__(self, p, a, d):
        assert a != d and is_prime(p) and p > 3
        self.K = GF(p)  # Definido como atributo para possível uso futuro

        # Convertendo a e d para elementos do campo finito K
        self.a = self.K(a)
        self.d = self.K(d)

        # Calculando A e B dentro do campo finito K
        A = 2 * (self.a + self.d) / (self.a - self.d)
        B = 4 / (self.a - self.d)

        self.alfa = A / (3 * B)
        self.s = B

        # Calculando a4 e a6 no campo K
        a4 = self.s**(-2) - 3 * self.alfa**2
        a6 = -self.alfa**3 - a4 * self.alfa

        self.EC = EllipticCurve(self.K, [a4, a6])

        # Pontos base convertidos para K
        self.Px = self.K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.Py = self.K(46316835694926478169428394003475163141307993866256225615783033603165251855960)

        self.L = ZZ(2**252 + 27742317777372353535851937790883648493)
        self.P = self.ed2ec(self.Px, self.Py)

        self.private_key = self.generate_private_key()

    def generate_private_key(self):
        return randint(1, self.L - 1)
    
    def generate_public_key(self):
        return self.private_key * self.P
    
    def ed2ec(self, x, y):
        if (x, y) == (0, 1):
            return self.EC(0)
        z = (1 + y) / (1 - y)
        w = z / x
        return self.EC(z / self.s + self.alfa, w / self.s)

    # Os métodos sign e verify permanecem os mesmos
    def sign(self, message):
        h = hashlib.sha512(message).digest()
        h_int = int.from_bytes(h, 'big') % self.L

        k = randint(1, self.L - 1)
        R = k * self.P

        r = int(R[0]) % self.L

        s = (k + h_int * self.private_key) % self.L

        return (r, s)
    
    def verify(self, message, signature, public_key):
        r, s = signature

        if not (1 <= r < self.L and 1 <= s < self.L):
            return False
        
        if not public_key in self.EC:
            return False
        
        h = hashlib.sha512(message).digest()
        h_int = int.from_bytes(h, 'big') % self.L

        R_prime = s * self.P - h_int * public_key

        return int(R_prime[0]) % self.L == r

E definindo a cifra e decifra baseado no que foi abordado anteriormente:

In [2]:
import numpy as np

def gen_keys(E, q):
    '''
    Gerar chaves pública e privada de ElGamal para a cifra e decifra a partir da ordem q
    '''
    sk = randint(1, q - 1)
    pk = sk * E.P
    return sk, pk

def encrypt(E,message,public_key):
    '''
    Escolher um número inteiro aleatório k no intervalo [1, L-1].
    Calcular o ponto C1 = k * P.
    Converter a mensagem m num ponto na curva elíptica
    Calcular o ponto C2 = m + k * pk.
    O texto cifrado é o par (C1, C2)..
    '''
    k = randint(1, E.L - 1)
    C1 = k * E.P
    m_point = encode_message(E, message)
    C2 = m_point + k * public_key    
    return (C1, C2)
    
def decrypt(E,ciphertext,private_key):
    '''
    Usar a chave privada sk para calcular m = C2 - sk * C1.
    Converter o ponto m de volta para a mensagem original
    '''
    C1, C2 = ciphertext
    m_point = C2 - (private_key * C1)
    
    return decode_message(E, m_point)

def is_quadratic_residue(x, p):
    """
    Verifica se x é um resíduo quadrático módulo p.
    """
    return legendre_symbol(x, p) == 1

def encode_message(E, message, l=8):
    """
    Converte uma mensagem num ponto na curva elíptica.
    """
    K = E.K
    q = K.order()
    k = q.nbits() 
    m_int = int.from_bytes(message, 'big')
    m_bits = m_int.bit_length()

    # Ajusta o tamanho da mensagem para k-1-l bits
    if m_bits > k - 1 - l:
        raise ValueError("Mensagem muito grande para o campo finito.")

    # Concatena a mensagem com l bits zero
    x = (m_int << l)  # Equivalente a m || 0^l

    # Tenta encontrar um ponto válido na curva
    for i in range(2**l):
        x_prime = x + i  
        x_prime = K(x_prime)  

        # Calcula x' = x^3 + a*x + b (equação da curva)
        y_squared = x_prime**3 + E.EC.a4() * x_prime + E.EC.a6()

        # Verifica se y_squared é um resíduo quadrático
        if is_quadratic_residue(y_squared, q):
            y = y_squared.sqrt()
            return E.EC(x_prime, y) 
            
    # Caso ultrapasse o limite de 2^l tentativas:
    raise ValueError("Não foi possível codificar a mensagem em um ponto da curva.")

def decode_message(E, point):
    """
    Converte um ponto na curva elíptica de volta para a mensagem original.
    """
    x = int(point[0]) 
    l = 8  # Número de bits usados para o padding
    m_int = x >> l  # Remove os últimos l bits
    m_bytes = m_int.to_bytes((m_int.bit_length() + 7) // 8, 'big')
    return m_bytes

#### **Exemplo de uso:**

In [3]:
# Exemplo de uso
if __name__ == "__main__":
    # Parâmetros da curva Ed25519
    p = 2**255 - 19
    a = -1
    d = -121665 / 121666

    # Instancia a curva
    E = EcDSA_Ed25519(p, a, d)

    # Mensagem de exemplo
    message = b"Hello, Ed25519!"

    # Codifica a mensagem em um ponto na curva
    try:
        m_point = encode_message(E, message)
        print(f"Mensagem codificada como ponto: {m_point}")
    except ValueError as e:
        print(e)

    # Decodifica o ponto de volta para a mensagem
    decoded_message = decode_message(E, m_point)
    print(f"Mensagem decodificada: {decoded_message.decode()}")

    # Teste de cifra e decifra
    sk, pk = gen_keys(E, E.L)
    ciphertext = encrypt(E, message, pk)
    print(f"Texto cifrado: {ciphertext}")

    decrypted_message = decrypt(E, ciphertext, sk)
    print(f"Mensagem decifrada: {decrypted_message.decode()}")

Mensagem codificada como ponto: (96231036770510887582514965757268926721 : 9327398782287318265056166936429671725248326146653680186023877059085667908223 : 1)
Mensagem decodificada: Hello, Ed25519!
Texto cifrado: ((2523425871936966218282680275620987981300259910143950160513254210741988079054 : 38931871572636621283414838541043388201152835068140378724821616721798198669985 : 1), (33384531941930146788726174476199750345194765044272099532160194890117511353208 : 46359258041945112270863595375989683816157315044804958676812483601286740730327 : 1))
Mensagem decifrada: Hello, Ed25519!


------

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

### Implementação

In [4]:
def g(r,message):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    g.update(r_bytes)
    final_hash = g.digest()  # Truncar para o tamanho de x
    while len(final_hash) < len(message):
        g = hashlib.sha512()
        g.update(r_bytes)
        final_hash += g.digest()
    return final_hash[:len(message)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    full_hash = h.digest()[:64 // 8]
    return full_hash

In [5]:
# Criar uma seed fixa
import random
random.seed(b"10")

def f_p(public_key,r,rlinha):
    k = random.randint(1, E.L - 1)
    C1 = k * E.P
    m_point = encode_message(E, message)
    C2 = m_point + k * public_key
    if not (C1 in E and C2 in E):
        raise ValueError("C1 e C2 devem ser pontos válidos na curva elíptica.")
    return (C1, C2)

In [6]:
def encrypt_FO(E,message,public_key):
    r = randint(1,E.L)
    print("message:",message)
    y = bytes(a ^^ b for a, b in zip(message, g(r,message)))
    rlinha = h(r,y)
    c = f_p(public_key,r,rlinha)
    return (y,c)

def decrypt_FO(E,ciphertext,private_key):
    r = decrypt(E,ciphertext,private_key)
    rlinha = h(r,y)
    if c != f_p(r,rlinha):
        print("Absurdo!")
        return
    else:
        return bytes(a ^^ b for a, b in zip(y, g(r)))

In [7]:
message = b"Ola!!!!"
# Codifica a mensagem em um ponto na curva
try:
    m_point = encode_message(E, message)
    print(f"Mensagem codificada como ponto: {m_point}")
except ValueError as e:
    print(e)

# Decodifica o ponto de volta para a mensagem
decoded_message = decode_message(E, m_point)
print(f"Mensagem decodificada: {decoded_message.decode()}")

# Teste de cifra e decifra
sk, pk = gen_keys(E, E.L)
ciphertext = encrypt_FO(E, message, pk) 
print(f"Texto cifrado: {ciphertext}")

decrypted_message = decrypt_FO(E, ciphertext, sk)
print(f"Mensagem decifrada: {decrypted_message.decode()}")

Mensagem codificada como ponto: (5723056021398692096 : 23117725897293925760354401963284524461283169530651030787792243700936166073628 : 1)
Mensagem decodificada: Ola!!!!
message: b'Ola!!!!'


TypeError: argument of type 'EcDSA_Ed25519' is not iterable

## “Oblivious Transfer” $\,\kappa$-out-of-$n\,$